## neural ODE with exogenous inputs applied on CED and CCT benchmarks

* includes exogenous inputs
* includes the introduced normalization term

In [18]:
import enum
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.auto import tqdm

from torchdiffeq import odeint #includes exogenous inputs

class LatentODEfunc(nn.Module):

    def __init__(self, latent_dim=4, nhidden=20, udim=None):
        super(LatentODEfunc, self).__init__()
        self.elu = nn.ELU(inplace=True)
        udim_val = 0 if udim is None else udim
        self.fc1 = nn.Linear(latent_dim + udim_val, nhidden)
        self.fc2 = nn.Linear(nhidden, nhidden)
        self.fc3 = nn.Linear(nhidden, latent_dim)
        self.nfe = 0

    def forward(self, t, x, u=None):
        self.nfe += 1
        if u is not None: #append the input to the state
            x = torch.cat([x,u[:,None] if u.ndim==1 else u],dim=1)
        out = self.fc1(x)
        out = self.elu(out)
        out = self.fc2(out)
        out = self.elu(out)
        out = self.fc3(out)
        return out


class RecognitionRNN(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=25, nbatch=1):
        super(RecognitionRNN, self).__init__()
        self.nhidden = nhidden
        self.nbatch = nbatch
        self.i2h = nn.Linear(obs_dim + nhidden, nhidden)
        self.h2o = nn.Linear(nhidden, latent_dim * 2)

    def forward(self, x, h):
        combined = torch.cat((x, h), dim=1)
        h = torch.tanh(self.i2h(combined))
        out = self.h2o(h)
        return out, h

    def initHidden(self):
        return torch.zeros(self.nbatch, self.nhidden)


class Decoder(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=20):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(latent_dim, nhidden)
        self.fc2 = nn.Linear(nhidden, obs_dim)

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out


class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


def log_normal_pdf(x, mean, logvar):
    const = torch.from_numpy(np.array([2. * np.pi])).float().to(x.device)
    const = torch.log(const)
    return -.5 * (const + logvar + (x - mean) ** 2. / torch.exp(logvar))


def normal_kl(mu1, lv1, mu2, lv2):
    v1 = torch.exp(lv1)
    v2 = torch.exp(lv2)
    lstd1 = lv1 / 2.
    lstd2 = lv2 / 2.

    kl = lstd2 - lstd1 + ((v1 + (mu1 - mu2) ** 2.) / (2. * v2)) - .5
    return kl

def get_train_val_test(dataset):
    import deepSI
    from deepSI import System_data_list
    if dataset=='CED':
        data_full = deepSI.datasets.CED()
        train = System_data_list([data_i[:300] for data_i in data_full])
        test = System_data_list([data_i[300:] for data_i in data_full])
        val = System_data_list([t[:100] for t in test])
    elif dataset=='CCT':
        train, test = deepSI.datasets.Cascaded_Tanks()
        val, test = test[:len(test)//2], test
    return train, val, test

def get_torch_data(dataset, device=torch.device('cpu')):
    train, val, test = get_train_val_test(dataset)
    from deepSI.system_data import System_data_norm
    norm = System_data_norm() #normalization
    norm.fit(train)
    train, val, test = [norm.transform(t) for t in [train, val, test]]

    if dataset=='CCT':
        samp_trajs = train.y[None,:,None]
        u_samp_trajs = train.u[None,:,None]
        orig_trajs = test.y[None,:,None]
        u_orig_trajs = test.u[None,:,None]
        val_trajs = val.y[None,:,None]
        u_val_trajs = val.u[None,:,None]

        sample_time = 4. #seconds
        samp_ts = np.arange(len(train.y))*sample_time

    elif dataset=='CED':
        samp_trajs = np.array([ti.y[:,None] for ti in train])
        u_samp_trajs =  np.array([ti.u for ti in train])
        orig_trajs = np.array([ti.y[:,None] for ti in test])
        u_orig_trajs = np.array([ti.u for ti in test])
        val_trajs = np.array([ti.y[:,None] for ti in val])
        u_val_trajs = np.array([ti.u[:,None] for ti in val])

        sample_time = 1/50 #seconds
        samp_ts = np.arange(len(train[0]))*sample_time

    orig_trajs = torch.from_numpy(orig_trajs).float().to(device) #samples, time
    u_orig_trajs = torch.from_numpy(u_orig_trajs).float().to(device)
    samp_trajs = torch.from_numpy(samp_trajs).float().to(device)
    u_samp_trajs = torch.from_numpy(u_samp_trajs).float().to(device)
    samp_ts = torch.from_numpy(samp_ts).float().to(device)
    u_val_trajs = torch.from_numpy(u_val_trajs).float().to(device)
    val_trajs = torch.from_numpy(val_trajs).float().to(device)
    
    return orig_trajs, u_orig_trajs, samp_trajs, u_samp_trajs, samp_ts, \
            u_val_trajs, val_trajs, sample_time, norm

In [19]:
dataset='CED'
method = 'rk4' #exogenous inputs only implemented for Euler, RK4 and midpoint.
device = torch.device('cpu')
train_dir = './models-neural-ode/'
niters = 20000

#given by latent_ODE:
nhidden = 20
rnn_nhidden = 25
obs_dim = 1
noise_std = 0.1

orig_trajs, u_orig_trajs, samp_trajs, u_samp_trajs, samp_ts, u_val_trajs, val_trajs, sample_time, norm = \
        get_torch_data(dataset, device=device)

if dataset=='CCT':
    dttau = 0.032
    latent_dim = 2
    
elif dataset=='CED':
    dttau = 0.12
    latent_dim = 3

samp_ts *= dttau/sample_time #dt /dt * dttau = dttau

In [1]:
for I in range(1,100):
    
    # create model
    print(f'I={I}')
    udim = 1 if u_orig_trajs.ndim==2 else u_orig_trajs.shape[-1]
    func = LatentODEfunc(latent_dim, nhidden, udim=udim).to(device)
    rec = RecognitionRNN(latent_dim, obs_dim, rnn_nhidden, samp_trajs.shape[0]).to(device)
    dec = Decoder(latent_dim, obs_dim, nhidden).to(device)
    params = (list(func.parameters()) + list(dec.parameters()) + list(rec.parameters()))
    optimizer = optim.Adam(params, lr=0.01)
    loss_meter = RunningAverageMeter()

    try: #for KeyboardInterrupt stopping of iterations
        best_val = float('inf')
        for itr in tqdm(range(1, niters + 1)):
            optimizer.zero_grad()
            # backward in time to infer q(z_0)
            h = rec.initHidden().to(device)
            for t in reversed(range(samp_trajs.size(1))):
                obs = samp_trajs[:, t, :]
                out, h = rec.forward(obs, h)
            qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
            epsilon = torch.randn(qz0_mean.size()).to(device)
            z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

            # forward in time and solve ode for reconstructions
            pred_z = odeint(func, z0, samp_ts, u=u_samp_trajs, method=method).permute(1, 0, 2)
            pred_x = dec(pred_z)

            # compute loss
            noise_std_ = torch.zeros(pred_x.size()).to(device) + noise_std
            noise_logvar = 2. * torch.log(noise_std_).to(device)
            logpx = log_normal_pdf(
                samp_trajs, pred_x, noise_logvar).sum(-1).sum(-1)
            pz0_mean = pz0_logvar = torch.zeros(z0.size()).to(device)
            analytic_kl = normal_kl(qz0_mean, qz0_logvar,
                                    pz0_mean, pz0_logvar).sum(-1)
            loss = torch.mean(-logpx + analytic_kl, dim=0)
            loss.backward()
            optimizer.step()
            loss_meter.update(loss.item())


            if has_val and itr%10==0:

                h = rec.initHidden().to(device)
                for t in reversed(range(val_trajs.size(1))):
                    obs = val_trajs[:, t, :]
                    out, h = rec.forward(obs, h)
                qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
                epsilon = torch.randn(qz0_mean.size()).to(device)
                z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

                # forward in time and solve ode for reconstructions
                pred_z = odeint(func, z0, samp_ts[:u_val_trajs.shape[1]], u=u_val_trajs, method=method).permute(1, 0, 2)
                pred_x = dec(pred_z)
                val_trajs_p = pred_x.detach()
                RMS = torch.mean((val_trajs_p - val_trajs)**2).item()**0.5
                print('Iter: {}, running avg elbo: {:.4f} rms val {:.4f}'.format(itr, -loss_meter.avg, RMS))
                if RMS<best_val:
                    best_val = RMS
                    if train_dir is not None:
                        ckpt_path = os.path.join(train_dir, f'ckpt-best-{dataset}-{I}.pth')
                        torch.save({
                            'func':func,
                            'rec':rec,
                            'dec':dec,
                            'func_state_dict': func.state_dict(),
                            'rec_state_dict': rec.state_dict(),
                            'dec_state_dict': dec.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'orig_trajs': orig_trajs,
                            'samp_trajs': samp_trajs,
                            'samp_ts': samp_ts,
                        }, ckpt_path)
                        print('Stored ckpt at {} {}'.format(ckpt_path, RMS))

    except KeyboardInterrupt:
        print(f'stopped by keyboard I={I}')
        break
    if train_dir is not None:
        ckpt_path = os.path.join(train_dir, f'ckpt-last-{dataset}-{I}.pth')
        torch.save({
            'func':func,
            'rec':rec,
            'dec':dec,
            'func_state_dict': func.state_dict(),
            'rec_state_dict': rec.state_dict(),
            'dec_state_dict': dec.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'orig_trajs': orig_trajs,
            'samp_trajs': samp_trajs,
            'samp_ts': samp_ts,
        }, ckpt_path)
        print('Stored ckpt at {}'.format(ckpt_path))
    print('Training complete after {} iters.'.format(itr))

time_nor 125.0
orig_trajs.shape=torch.Size([1, 1024, 1])
I=4


  0%|          | 0/20000 [00:00<?, ?it/s]

Iter: 10, running avg elbo: -51177.3670 rms val 1.0668
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 1.0667796057773842
Iter: 20, running avg elbo: -50993.7049 rms val 1.0015
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 1.0015329294109396
Iter: 30, running avg elbo: -50317.7481 rms val 0.9515
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.9515247356268599
Iter: 40, running avg elbo: -48930.5600 rms val 0.8313
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.8313206248289656
Iter: 50, running avg elbo: -45888.7856 rms val 0.4249
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.4248858144180097
Iter: 60, running avg elbo: -42291.4151 rms val 0.4040
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.404037982406307
Iter: 70, running avg elbo: -38787.3286 rms val 0.4238
Iter: 80, running avg elbo: -35528.9265 rms val 0.3876
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.38756243833366977
Iter: 90, running avg elbo: -32532.4848 rms val 0.3760
Stored ckpt at ./traindir/ckpt-best-CCT-4.pth 0.37

Iter: 1390, running avg elbo: 1160.9287 rms val 0.5259
Iter: 1400, running avg elbo: 1161.8790 rms val 0.5325
Iter: 1410, running avg elbo: 1166.3737 rms val 0.5817
Iter: 1420, running avg elbo: 1168.3073 rms val 0.5802
Iter: 1430, running avg elbo: 1173.1004 rms val 0.5549
Iter: 1440, running avg elbo: 1173.7636 rms val 0.5893
Iter: 1450, running avg elbo: 1172.1159 rms val 0.6217
Iter: 1460, running avg elbo: 1173.3000 rms val 0.5462
Iter: 1470, running avg elbo: 1173.0114 rms val 0.5426
Iter: 1480, running avg elbo: 1173.9831 rms val 0.6753
Iter: 1490, running avg elbo: 1174.2033 rms val 0.6400
Iter: 1500, running avg elbo: 1176.5626 rms val 0.6285
Iter: 1510, running avg elbo: 1178.1520 rms val 0.6123
Iter: 1520, running avg elbo: 1177.1997 rms val 0.6427
Iter: 1530, running avg elbo: 1178.5838 rms val 0.6040
Iter: 1540, running avg elbo: 1181.6308 rms val 0.5737
Iter: 1550, running avg elbo: 1184.8932 rms val 0.6779
Iter: 1560, running avg elbo: 1185.3417 rms val 0.6815
Iter: 1570

Iter: 2880, running avg elbo: 1277.8762 rms val 1.0116
Iter: 2890, running avg elbo: 1278.1125 rms val 0.9926
Iter: 2900, running avg elbo: 1279.1617 rms val 0.9939
Iter: 2910, running avg elbo: 1279.1687 rms val 1.0135
Iter: 2920, running avg elbo: 1279.4020 rms val 1.0020
Iter: 2930, running avg elbo: 1279.0251 rms val 1.0183
Iter: 2940, running avg elbo: 1280.7246 rms val 1.0015
Iter: 2950, running avg elbo: 1279.0473 rms val 1.0046
Iter: 2960, running avg elbo: 1280.5211 rms val 1.0147
Iter: 2970, running avg elbo: 1279.7682 rms val 0.9807
Iter: 2980, running avg elbo: 1280.0382 rms val 1.0202
Iter: 2990, running avg elbo: 1279.3830 rms val 1.0097
Iter: 3000, running avg elbo: 1280.4274 rms val 0.9743
Iter: 3010, running avg elbo: 1279.4624 rms val 1.0309
Iter: 3020, running avg elbo: 1279.9764 rms val 0.9863
Iter: 3030, running avg elbo: 1281.3939 rms val 1.0271
Iter: 3040, running avg elbo: 1282.7464 rms val 1.0143
Iter: 3050, running avg elbo: 1283.7400 rms val 1.0188
Iter: 3060

...

Iter: 2090, running avg elbo: 1299.5590 rms val 0.1126
Iter: 2100, running avg elbo: 1300.9139 rms val 0.1128
Iter: 2110, running avg elbo: 1300.7438 rms val 0.1154
Iter: 2120, running avg elbo: 1302.1394 rms val 0.1137
Iter: 2130, running avg elbo: 1304.1660 rms val 0.1122
Iter: 2140, running avg elbo: 1305.8020 rms val 0.1134
Iter: 2150, running avg elbo: 1306.3271 rms val 0.1155
Iter: 2160, running avg elbo: 1307.6951 rms val 0.1084
Iter: 2170, running avg elbo: 1308.6302 rms val 0.1057
Iter: 2180, running avg elbo: 1309.0322 rms val 0.1060
Iter: 2190, running avg elbo: 1310.5590 rms val 0.1080
Iter: 2200, running avg elbo: 1312.0374 rms val 0.1086
Iter: 2210, running avg elbo: 1312.9469 rms val 0.1055
Iter: 2220, running avg elbo: 1314.0586 rms val 0.1125
Iter: 2230, running avg elbo: 1315.5541 rms val 0.1121
Iter: 2240, running avg elbo: 1315.8016 rms val 0.1130
Iter: 2250, running avg elbo: 1316.9970 rms val 0.1189
Iter: 2260, running avg elbo: 1315.4903 rms val 0.1054
Iter: 2270

Iter: 3570, running avg elbo: 1337.7816 rms val 0.1127
Iter: 3580, running avg elbo: 1338.4687 rms val 0.1231
Iter: 3590, running avg elbo: 1337.9595 rms val 0.1027
Stored ckpt at ./traindir/ckpt-best-CCT-16.pth 0.1026921582599115
Iter: 3600, running avg elbo: 1338.4296 rms val 0.1178
Iter: 3610, running avg elbo: 1339.0071 rms val 0.1043
Iter: 3620, running avg elbo: 1338.9986 rms val 0.1142
Iter: 3630, running avg elbo: 1339.8474 rms val 0.1102
Iter: 3640, running avg elbo: 1338.9809 rms val 0.1023
Stored ckpt at ./traindir/ckpt-best-CCT-16.pth 0.10229337868630384
Iter: 3650, running avg elbo: 1337.6102 rms val 0.1097
Iter: 3660, running avg elbo: 1338.6129 rms val 0.1135
Iter: 3670, running avg elbo: 1339.3811 rms val 0.1125
Iter: 3680, running avg elbo: 1339.9099 rms val 0.1137
Iter: 3690, running avg elbo: 1340.4703 rms val 0.1161
Iter: 3700, running avg elbo: 1339.7545 rms val 0.1077
Iter: 3710, running avg elbo: 1339.2550 rms val 0.1236
Iter: 3720, running avg elbo: 1336.1447 rm

Iter: 4970, running avg elbo: 1349.9643 rms val 0.1034
Iter: 4980, running avg elbo: 1350.3270 rms val 0.1122
Iter: 4990, running avg elbo: 1349.7459 rms val 0.1011
Iter: 5000, running avg elbo: 1351.0183 rms val 0.0987
Iter: 5010, running avg elbo: 1351.9899 rms val 0.1009
Iter: 5020, running avg elbo: 1352.3907 rms val 0.1142
Iter: 5030, running avg elbo: 1351.0589 rms val 0.0956
Iter: 5040, running avg elbo: 1351.9214 rms val 0.0967
Iter: 5050, running avg elbo: 1352.2198 rms val 0.1032
Iter: 5060, running avg elbo: 1352.8950 rms val 0.1048
Iter: 5070, running avg elbo: 1353.3302 rms val 0.0942
Stored ckpt at ./traindir/ckpt-best-CCT-16.pth 0.09415224328111091
Iter: 5080, running avg elbo: 1352.4209 rms val 0.1156
Iter: 5090, running avg elbo: 1350.0982 rms val 0.0951
Iter: 5100, running avg elbo: 1350.4310 rms val 0.0980
Iter: 5110, running avg elbo: 1351.1336 rms val 0.0958
Iter: 5120, running avg elbo: 1350.5805 rms val 0.0954
Iter: 5130, running avg elbo: 1350.0334 rms val 0.099

Iter: 6440, running avg elbo: 1365.1585 rms val 0.0987
Iter: 6450, running avg elbo: 1365.0109 rms val 0.0968
Iter: 6460, running avg elbo: 1365.2277 rms val 0.1051
Iter: 6470, running avg elbo: 1364.3244 rms val 0.0943
Iter: 6480, running avg elbo: 1364.1820 rms val 0.0995
Iter: 6490, running avg elbo: 1364.4675 rms val 0.1014
Iter: 6500, running avg elbo: 1364.5875 rms val 0.0968
Iter: 6510, running avg elbo: 1365.2868 rms val 0.0978
Iter: 6520, running avg elbo: 1365.5060 rms val 0.0989
Iter: 6530, running avg elbo: 1365.3792 rms val 0.0982
Iter: 6540, running avg elbo: 1365.8066 rms val 0.0998
Iter: 6550, running avg elbo: 1365.7925 rms val 0.1000
Iter: 6560, running avg elbo: 1363.2804 rms val 0.0970
Iter: 6570, running avg elbo: 1363.4357 rms val 0.1069
Iter: 6580, running avg elbo: 1362.8779 rms val 0.0992
Iter: 6590, running avg elbo: 1363.8766 rms val 0.0961
Iter: 6600, running avg elbo: 1364.2456 rms val 0.1043
Iter: 6610, running avg elbo: 1363.4231 rms val 0.1017
Iter: 6620

Iter: 7930, running avg elbo: 1373.5294 rms val 0.1204
Iter: 7940, running avg elbo: 1372.5968 rms val 0.1216
Iter: 7950, running avg elbo: 1372.1905 rms val 0.1214
Iter: 7960, running avg elbo: 1373.1193 rms val 0.1185
Iter: 7970, running avg elbo: 1373.8138 rms val 0.1209
Iter: 7980, running avg elbo: 1372.9344 rms val 0.1169
Iter: 7990, running avg elbo: 1373.4126 rms val 0.1178
Iter: 8000, running avg elbo: 1373.7835 rms val 0.1168
Iter: 8010, running avg elbo: 1374.5122 rms val 0.1157
Iter: 8020, running avg elbo: 1374.8237 rms val 0.1176
Iter: 8030, running avg elbo: 1374.5830 rms val 0.1202
Iter: 8040, running avg elbo: 1373.6409 rms val 0.1161
Iter: 8050, running avg elbo: 1373.3787 rms val 0.1206
Iter: 8060, running avg elbo: 1372.8899 rms val 0.1175
Iter: 8070, running avg elbo: 1373.0532 rms val 0.1199
Iter: 8080, running avg elbo: 1373.4898 rms val 0.1216
Iter: 8090, running avg elbo: 1373.2810 rms val 0.1184
Iter: 8100, running avg elbo: 1373.9539 rms val 0.1179
Iter: 8110